In [ ]:
#Question 1: Maze Problem 
# Import the heapq module for priority queue operations
import heapq

# Function to find neighboring cells in the maze
def neighbors(maze, current):
    # Unpack the current cell coordinates
    x, y = current
    # Define possible neighbors (up, down, left, right)
    possible_neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]
    # Return only valid neighbors (within maze bounds and not an obstacle)
    return [(nx, ny) for nx, ny in possible_neighbors if 0 <= nx < len(maze) and 0 <= ny < len(maze[0]) and maze[nx][ny] == 0]

# Heuristic function for A* algorithm (Manhattan distance)
def heuristic(a, b):
    # Unpack the coordinates
    (x1, y1) = a
    (x2, y2) = b
    # Return the Manhattan distance
    return abs(x1 - x2) + abs(y1 - y2)

# Function to reconstruct the path from start to goal using came_from dictionary
def reconstruct_path(came_from, start, goal):
    # Start from the goal
    current = goal
    path = []
    # Work our way back to the start
    while current != start:
        path.append(current)
        current = came_from[current]
    # Add the start to the path
    path.append(start)
    # Reverse the path so it's in the correct order and return it
    return path[::-1]

# A* algorithm to find the shortest path from start to multiple goals
def a_star_search(maze, start, goals):
    # Initialize the frontier with the start position
    frontier = []
    heapq.heappush(frontier, (0, start))
    # Initialize an empty dictionary to store where we came from
    came_from = {}
    # Initialize an empty dictionary to store the cost to reach each cell
    cost_so_far = {}
    # The start cell has no cell that we came from
    came_from[start] = None
    # The cost to reach the start cell is 0
    cost_so_far[start] = 0
    # Initialize an empty list to store the goals that have been found
    found_goals = []

    # Continue until there are no more cells to explore
    while frontier:
        # Get the cell with the highest priority (lowest cost) from the frontier
        _, current = heapq.heappop(frontier)

        # If the current cell is a goal and we haven't found it yet, add it to the list of found goals
        if current in goals and current not in found_goals:
            found_goals.append(current)
            # If we've found all goals, stop
            if len(found_goals) == len(goals):
                break

        # Explore the neighbors of the current cell
        for next in neighbors(maze, current):
            # Calculate the cost to reach the next cell
            new_cost = cost_so_far[current] + 1
            # If we haven't visited the next cell yet, or if we've found a cheaper path to it, update the cost and priority
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, current)
                heapq.heappush(frontier, (priority, next))
                came_from[next] = current

    # After exploring the maze, reconstruct the paths from the start to each goal
    paths = [reconstruct_path(came_from, start, goal) if goal in came_from else [] for goal in goals]
    # Return the paths
    return paths

# Example usage:
maze = [
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
    [0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
]

start = (10, 5)
goals = [(0, 5), (4, 2)]

paths = a_star_search(maze, start, goals)
print("Paths to goals:", paths)





In [ ]:
#Question 2: Tic-Tac-Toe
import json

# Function to generate all possible states of the game
def generate_states():
    # Initialize a set to store the states
    states = set()
    # Start the recursive function with the initial state
    generate_recursive("X---X---O", states)
    # Convert the set to a list
    states_list = list(states)
    # Create a dictionary with each state and its child states
    states_dict = {state: get_child_states(state) for state in states_list}
    return states_dict

# Recursive function to generate all possible states
def generate_recursive(state, states):
    # If the state is not already in the set
    if state not in states:
        # Add the state to the set
        states.add(state)
        # For each position in the state
        for i in range(len(state)):
            # If the position is empty
            if state[i] == "-":
                # Generate a new state with an 'X' in the position
                new_state = state[:i] + "X" + state[i+1:]
                # Call the function recursively with the new state
                generate_recursive(new_state, states)
                # Generate a new state with an 'O' in the position
                new_state = state[:i] + "O" + state[i+1:]
                # Call the function recursively with the new state
                generate_recursive(new_state, states)

# Function to get all child states of a given state
def get_child_states(state):
    # Initialize a list to store the child states
    child_states = []
    # For each position in the state
    for i in range(len(state)):
        # If the position is empty
        if state[i] == "-":
            # Generate a new state with an 'X' in the position and add it to the list
            new_state_x = state[:i] + "X" + state[i+1:]
            child_states.append(new_state_x)
            # Generate a new state with an 'O' in the position and add it to the list
            new_state_o = state[:i] + "O" + state[i+1:]
            child_states.append(new_state_o)
    return child_states

# Function to convert a state string to a 3x3 grid
def state_to_3x3_grid(state):
    # Convert the state to a 3x3 grid
    return [list(state[i:i+3]) for i in range(0, len(state), 3)]

# Function to save the states dictionary to a JSON file
def save_to_json(states_dict, filename="tic_tac_toe_states.json"):
    # Format the states for saving
    formatted_states = {state: state_to_3x3_grid(state) for state in states_dict.keys()}
    # Open the file in write mode
    with open(filename, 'w') as file:
        # Dump the dictionary to the file
        json.dump(formatted_states, file, indent=2)

# Function to print a Tic-Tac-Toe board
def print_board(state):
    # For each row in the state
    for row in state:
        # Print the row
        print(row)

# Generate all possible states
tic_tac_toe_states = generate_states()
# Save the states to a JSON file
save_to_json(tic_tac_toe_states)

# Print a message indicating that the states have been generated and saved
print("Tic-Tac-Toe states generated and saved to 'tic_tac_toe_states.json'.")
# Print an example of a Tic-Tac-Toe board
print("Here is an example of a Tic-Tac-Toe board:")
print_board(state_to_3x3_grid("X---X---O"))


In [ ]:
#Question 3: MNIST
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data to a 1D array
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Build the model
model = Sequential([
    Flatten(input_shape=(784,)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')


In [ ]:
#Question 4: California Housing Regression 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.datasets import fetch_california_housing

# Function to load and split the California housing dataset
def load_data(version="large", test_split=0.2, seed=113):
    # Fetch the California housing dataset
    data = fetch_california_housing()
    
    # Split the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(data.data, data.target, test_size=test_split, random_state=seed)
    
    return (x_train, y_train), (x_test, y_test)

# Function to preprocess data using StandardScaler
def preprocess_data(x_train, x_test):
    # Initialize StandardScaler
    scaler = StandardScaler()
    
    # Scale the training and testing data
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    return x_train_scaled, x_test_scaled

# Function to build a regression model using TensorFlow/Keras
def build_regression_model(input_shape):
    # Create a Sequential model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    
    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    
    return model

# Function to train and evaluate the regression model
def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, epochs=50, batch_size=32):
    # Train the model
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Predict on the test set
    y_pred = model.predict(x_test).flatten()
    
    # Calculate Mean Squared Error and Mean Absolute Error
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    return mse, mae

# Main function
def main():
    # Load data
    (x_train, y_train), (x_test, y_test) = load_data()

    # Preprocess data
    x_train_scaled, x_test_scaled = preprocess_data(x_train, x_test)

    # Build and train the model
    model = build_regression_model(input_shape=x_train_scaled.shape[1])
    mse, mae = train_and_evaluate_model(model, x_train_scaled, y_train, x_test_scaled, y_test)

    # Display results
    print(f'Mean Squared Error (MSE): {mse:.2f}')
    print(f'Mean Absolute Error (MAE): {mae:.2f}')

# Run the main function if the script is executed directly
if __name__ == "__main__":
    main()
